In [115]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
base_url = 'https://ayna.az/'

In [107]:
def get_soup(url):

    while True:

        try:

            response=requests.get(url)

            html_ici=response.content

            soup=BeautifulSoup(html_ici, 'html.parser')

            return soup

        except:
            continue

In [7]:
def get_categories(url):
    cat = {}
    soup = get_soup(url)
    ul = soup.find('ul',{'class':"navbar-nav nav-menu"})
    
    for a in ul.find_all('a'):
        cat[a.text.strip('\n')] = a.get('href')
    return cat

In [8]:
print(get_categories(base_url))

{'İqtİsadİyyat': 'https://ayna.az/iqtisadiyyat', 'Xəbərlər ': 'https://ayna.az/xeberler', 'Hadisə ': 'https://ayna.az/xeberler/hadise', 'Sİyasət': 'https://ayna.az/siyaset', 'Cəmİyyət': 'https://ayna.az/cemiyyet', 'Mədənİyyət': 'https://ayna.az/medeniyyet', 'Aktual': 'https://ayna.az/aktual', 'Dünya': 'https://ayna.az/dunya', 'İdman': 'https://ayna.az/idman', 'Tarİx': 'https://ayna.az/tarix', 'Yazarlar ': 'https://ayna.az/yazarlar', 'Elçin Şıxlı ': 'https://ayna.az/yazarlar/elcin-sixli', 'İbrahim Nəbioğlu ': 'https://ayna.az/yazarlar/ibrahim-nebioglu', 'Arif Əliyev ': 'https://ayna.az/yazarlar/arif-eliyev', 'Qulu Məhərrəmli ': 'https://ayna.az/yazarlar/qulu-meherremli', 'Maraqlı ': 'https://ayna.az/maraqli', 'Reklam ': 'https://ayna.az/maraqli/reklam'}


In [92]:
def get_page(url):

    n = 2
    urls = []
    
    while n<20:
        new_url = url + f'?page={n}'
        soup = get_soup(new_url)
        li  = soup.find('li',{'class':'next'})

        if li == '':
            break
        
        urls.append(new_url)
        n+=1
    urls.insert(0,url)
    
    return urls

In [72]:
def get_news_url(url):
    soup = get_soup(url)
    table = soup.find_all('div',{'class':'card pr-card'})
    
    urls=[a.find('a').get('href') for a in table]
    
    return urls

In [77]:
get_news_url('https://ayna.az/iqtisadiyyat?page=2')[0]

'https://ayna.az/azeri-light-ucuzlasib-66188'

In [80]:
def get_news_text(url):
    soup = get_soup(url)
    
    article = soup.find('article',{'class':'full-post-article news-article'})
    
    return article.text

In [110]:
answer = {'category':[], 'text':[]}

In [111]:
categories = get_categories(base_url)

In [112]:
try:
    for name, url in categories.items():
        pages = get_page(url)

        for page in pages:
            news_urls = get_news_url(page)

            for new_url in news_urls:
                text = get_news_text(new_url)
                answer['category'].append(name)
                answer['text'].append(text)
            
        
    print(answer)
except:
    pass

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [113]:
df = pd.DataFrame(answer)
df

,category,text
0,İqtİsadİyyat,\nHüquqşünas: “Azərbaycanda banklara qarşı irə...
1,İqtİsadİyyat,\nAzərbaycan neftinin qiyməti 93 dollara yaxın...
2,İqtİsadİyyat,\n Neftin qiymətində artım qeydə alınıb \nDüny...
3,İqtİsadİyyat,\nİlham Əliyev: “Hədəfimiz qazıntı yanacağında...
4,İqtİsadİyyat,\nAzərbaycan Hökuməti ilə Birləşmiş Ərəb Əmirl...
...,...,...
3613,Reklam,\nFonda ümumilikdə ianə edilmiş məbləğ 2 milyo...
3614,Reklam,\nTəbriz küçəsi 108 ünvanında yerləşən yeni “B...
3615,Reklam,\n“e-Tabib”lə yoluxmanın qarşısını al!” tətbiq...
3616,Reklam,\nŞirkət təklif ediyi xidmət və məhsulların tə...


In [114]:
df.to_csv('Rasul_ayna_data.csv')